In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. Membaca dataset preprocessed
df = pd.read_csv('data/preprocessed_data_full_encoded_new_v3.csv')
cols_to_keep = ['age', 
                'height',
                'weight',
                'systolic', 
                'diastolic',
                'bmi',
                'map',
                'pulse_pressure',
                'gender',
                'cholesterol', 
                'gluc',
                'smoke', 
                'alco', 
                'active',
                'cardio'
                ]

df = df[cols_to_keep]

# 2. Pisahkan fitur (X) dan target (y)
X = df.drop('cardio', axis=1)
y = df['cardio']

# 3. Definisikan tipe fitur
numerical_features = ['age', 'height', 'weight', 'systolic', 'diastolic', 'bmi', 'map', 'pulse_pressure']
ordinal_features = ['cholesterol', 'gluc']
binary_features = ['gender', 'smoke', 'alco', 'active']

# 4. Membuat ColumnTransformer untuk scaling hanya pada fitur numerik.
# Fitur ordinal dan biner dibiarkan apa adanya menggunakan remainder='passthrough'
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

# 5. Split data menjadi train dan test (80% train dan 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Terapkan preprocessor ke data training dan testing
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Catatan: ColumnTransformer dengan remainder='passthrough' akan mengubah urutan kolom.
# Fitur yang discaling akan berada di awal, kemudian sisanya.
# Untuk mendapatkan nama kolom kembali (opsional), kita bisa menyusun ulang nama kolom sebagai berikut.
passthrough_features = [col for col in X.columns if col not in numerical_features]
transformed_feature_names = numerical_features + passthrough_features

# Jika ingin melihat DataFrame hasil transformasi, bisa dilakukan seperti berikut:
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=transformed_feature_names)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=transformed_feature_names)
print("Contoh data train setelah scaling:")
print(X_train_transformed_df.head())

# 7. Membangun stacking classifier
# Definisikan base models, misalnya Random Forest dan Gradient Boosting
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42))
]

# Meta model menggunakan Logistic Regression
meta_model = LogisticRegression()

# Membangun stacking classifier dengan cross validation 5-fold
stacking_model = StackingClassifier(estimators=base_models,
                                    final_estimator=meta_model,
                                    cv=5)

# 8. Melatih model stacking
stacking_model.fit(X_train_transformed, y_train)

# Melakukan prediksi pada data testing
y_pred = stacking_model.predict(X_test_transformed)

# Evaluasi kinerja model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)


Contoh data train setelah scaling:
        age    height    weight  systolic  diastolic       bmi       map  \
0 -0.986440  1.814601  0.339893 -0.410327   1.090325 -0.590914  0.388501   
1 -0.251381 -1.092418  0.580050 -0.410327  -0.172570  1.260746 -0.309744   
2  0.777700  0.228954 -0.140421  0.258276  -0.172570 -0.268049  0.039902   
3  1.218736  0.096817 -0.460631  1.595483   2.353220 -0.513939  2.133591   
4  0.042642 -0.828144  0.499998  2.264087   2.353220  0.997750  2.482190   

   pulse_pressure  gender  cholesterol  gluc  smoke  alco  active  
0       -1.388972     1.0          0.0   0.0    1.0   0.0     1.0  
1       -0.448610     0.0          2.0   2.0    0.0   0.0     1.0  
2        0.491751     1.0          0.0   0.0    1.0   0.0     1.0  
3        0.491751     1.0          0.0   0.0    0.0   0.0     1.0  
4        1.432113     0.0          0.0   0.0    0.0   0.0     0.0  
Accuracy: 0.8630068245684465

Classification Report:
               precision    recall  f1-score   

In [2]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Misal, kita sudah memiliki variabel berikut:
# X_train, y_train: data training asli
# stacking_model: model stacking yang sudah dilatih dari langkah sebelumnya

# Definisikan tipe fitur numerik yang perlu discaling
numerical_features = ['age', 'height', 'weight', 'systolic', 'diastolic', 'bmi', 'map', 'pulse_pressure']

# Buat preprocessor untuk scaling hanya pada fitur numerik
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'  # Fitur lainnya (ordinal dan biner) tetap tidak berubah
)

# Buat pipeline yang menggabungkan preprocessor dan stacking_model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', stacking_model)
])

# Latih pipeline menggunakan data training asli (X_train dan y_train)
pipeline.fit(X_train, y_train)

# Simpan pipeline ke file pickle
with open("data/pipeline_model.pkl", "wb") as f:
    pickle.dump(pipeline, f)

print("Pipeline telah disimpan dalam file 'pipeline_model.pkl'.")


Pipeline telah disimpan dalam file 'pipeline_model.pkl'.


In [5]:
import pandas as pd
import pickle

# --- 1. Definisikan 10 contoh data baru (raw) dengan nilai kategori yang sudah di-encode ---
# Di sini dibuat 10 baris, dengan kombinasi nilai yang diharapkan menghasilkan label 0 dan 1.
data_new = {
    'age':       [30, 35, 40, 45, 50, 60, 65, 70, 55, 68],      # Usia dalam tahun
    'height':    [175, 180, 170, 165, 160, 158, 155, 150, 165, 160], # Tinggi badan dalam cm
    'weight':    [68, 75, 72, 80, 85, 90, 95, 100, 78, 82], # Berat badan dalam kg
    'systolic':  [110, 115, 120, 125, 130, 140, 150, 160, 115, 135],  # Tekanan sistolik (mm/Hg)
    'diastolic': [70, 75, 80, 85, 90, 95, 100, 105, 75, 88],         # Tekanan diastolik (mm/Hg)
    # Nilai kategori sudah di-encode:
    # gender: 0 untuk nilai asli 1, 1 untuk nilai asli 2
    'gender':       [0, 0, 1, 0, 1, 1, 1, 1, 0, 1],
    # cholesterol: 0 untuk nilai asli 1, 1 untuk nilai asli 2, 2 untuk nilai asli 3
    'cholesterol':  [0, 0, 1, 1, 2, 2, 2, 2, 0, 2],
    # gluc: 0 untuk nilai asli 1, 1 untuk nilai asli 2, 2 untuk nilai asli 3
    'gluc':         [0, 0, 1, 1, 1, 2, 2, 2, 0, 2],
    # Nilai biner: hanya boleh 0 atau 1
    'smoke':        [0, 0, 0, 0, 1, 1, 1, 1, 0, 1],
    'alco':         [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
    'active':       [1, 1, 1, 1, 0, 0, 0, 0, 1, 0]
}

# Buat DataFrame dari data baru
new_df = pd.DataFrame(data_new)

# --- 2. Validasi Nilai Kategori ---
# Pastikan nilai kategori setelah encoding sesuai dengan yang diharapkan.
valid_gender      = new_df['gender'].isin([0, 1])
valid_cholesterol = new_df['cholesterol'].isin([0, 1, 2])
valid_gluc        = new_df['gluc'].isin([0, 1, 2])
valid_smoke       = new_df['smoke'].isin([0, 1])
valid_alco        = new_df['alco'].isin([0, 1])
valid_active      = new_df['active'].isin([0, 1])

if not (valid_gender.all() and valid_cholesterol.all() and valid_gluc.all() and 
        valid_smoke.all() and valid_alco.all() and valid_active.all()):
    raise ValueError("Terdapat nilai kategori yang tidak valid pada input data.")

# --- 3. Proses Fitur Turunan ---
# Hitung fitur turunan: BMI, MAP, dan pulse_pressure secara otomatis
new_df["bmi"] = round(new_df["weight"] / ((new_df["height"] / 100) ** 2), 2)
new_df["map"] = round((new_df["systolic"] + 2 * new_df["diastolic"]) / 3, 2)
new_df["pulse_pressure"] = new_df["systolic"] - new_df["diastolic"]

# --- 4. Sesuaikan Urutan Kolom ---
# Urutan kolom yang diharapkan oleh pipeline:
expected_order = [
    'age', 'height', 'weight', 'systolic', 'diastolic', 
    'bmi', 'map', 'pulse_pressure',
    'gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active'
]
new_df = new_df[expected_order]

# --- 5. Muat Pipeline dari File Pickle ---
with open("data/pipeline_model.pkl", "rb") as f:
    pipeline = pickle.load(f)

# --- 6. Lakukan Prediksi pada Data Baru ---
predictions = pipeline.predict(new_df)

print("Input Data Baru (setelah pembuatan fitur turunan dan validasi):")
print(new_df)
print("\nHasil Prediksi untuk 10 contoh data:")
# Cetak hasil prediksi, dengan asumsi label 0 = sehat dan label 1 = status kardio
print("Prediksi status cardio untuk data baru:", predictions)


Input Data Baru (setelah pembuatan fitur turunan dan validasi):
   age  height  weight  systolic  diastolic    bmi     map  pulse_pressure  \
0   30     175      68       110         70  22.20   83.33              40   
1   35     180      75       115         75  23.15   88.33              40   
2   40     170      72       120         80  24.91   93.33              40   
3   45     165      80       125         85  29.38   98.33              40   
4   50     160      85       130         90  33.20  103.33              40   
5   60     158      90       140         95  36.05  110.00              45   
6   65     155      95       150        100  39.54  116.67              50   
7   70     150     100       160        105  44.44  123.33              55   
8   55     165      78       115         75  28.65   88.33              40   
9   68     160      82       135         88  32.03  103.67              47   

   gender  cholesterol  gluc  smoke  alco  active  
0       0            0   